<a href="https://colab.research.google.com/github/SakshiSawant/SocialDistanceAnalyser/blob/main/Social_Distancing_Analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Connecting the Drive and making the Environment ready for the object detection**

In [ ]:
#Connecting the colab with google drive
from google.colab import drive  
drive.mount('/content/gdrive',force_remount=True)  

Mounted at /content/gdrive


In [ ]:
#To read file from drive in colab
!ln -s /content/gdrive/My\ Drive/ /mydrive 

In [ ]:
%cd /mydrive/yolov4   

/content/gdrive/My Drive/yolov4


In [ ]:
#Neural Network framework 
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


In [ ]:
%cd darknet

/content/gdrive/My Drive/yolov4/darknet


In [ ]:
!chmod +x ./darknet

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


# **Testing the YOLOv4 Algorithm on sample images and videos**

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make 

chmod +x *.sh


In [ ]:
#Downloading the weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
# Defining the functions to upload and download files
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)) 
  plt.show()

# use this to upload files
def upload(): 
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
# run darknet detection on test images
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/person.jpg           

In [ ]:
# show image using our helper function 
imShow('predictions.jpg') 

In [ ]:
# Uploading the image
%cd ..
upload()
%cd darknet 

In [ ]:
# Detecting the objects of the image
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights ../people.jfif
imShow('predictions.jpg') 

In [ ]:
upload()  

In [ ]:
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show test.mp4 -i 0 -out_filename results.avi

In [ ]:
# download the video with detections shown 
download('results.avi') 

# **Analyzing the distance between the people**

In [ ]:
!make 

chmod +x *.sh


In [ ]:
!chmod +x ./darknet

In [ ]:
#Checking the files
%ls 

 3rdparty/                image_yolov3.sh*         src/
 app.py                   image_yolov4.sh*         static/
 backup/                  include/                 templates/
 build/                   json_mjpeg_streams.sh*   test2_output.avi
 build.ps1                libdarknet.so            test.mp4
 camera.py                LICENSE                  Untitled
 cfg/                     Makefile                'Untitled (1)'
 cmake/                   net_cam_v3.sh*           uselib
 CMakeLists.txt           net_cam_v4.sh*           vcpkg.json
 darknet*                 obj/                     video_yolov3.sh*
 DarknetConfig.cmake.in   predictions.jpg          video_yolov4.sh*
 darknet_images.py        __pycache__/             vid.mp4
 darknet.py               README.md                yolov4.weights
 darknet_video.py         results/
 data/                    scripts/


In [ ]:
%cd ..
 

/content/gdrive/My Drive/yolov4


In [ ]:
%cd darknet

/content/gdrive/My Drive/yolov4/darknet


In [ ]:
%cd /mydrive/yolov4 

/content/gdrive/My Drive/yolov4


In [ ]:
%cd darknet

/content/gdrive/My Drive/yolov4/darknet


In [ ]:
#Importing the libraries

In [ ]:
from ctypes import *
import math
import random
import os
import cv2
import numpy as np
import time
import darknet
from itertools import combinations

In [ ]:
def is_close(p1, p2):
    """
    Calculate Euclidean Distance between two points
    p1, p2 = two points for calculating Euclidean Distance
    dst = Euclidean Distance between two 2d points
    """
    dst = math.sqrt(p1**2 + p2**2)
    return dst 

In [ ]:
def convertBack(x, y, w, h): 
    """
    Converts center coordinates to rectangle coordinates
    x, y = midpoint of bbox
    w, h = width, height of the bbox
    xmin, ymin, xmax, ymax
    """
    xmin = int(round(x - (w / 2)))
    xmax = int(round(x + (w / 2)))
    ymin = int(round(y - (h / 2)))
    ymax = int(round(y + (h / 2)))
    return xmin, ymin, xmax, ymax


In [ ]:
def cvBoxes(detections, img):
    """
    detections = total detections in one frame
    img = image from detect_image method of darknet
    img with bounding box
    """

    # 3. Filtering the person class from detections and get bounding box centroid for each person detection

    if len(detections) > 0:  						# At least 1 detection in the image and check detection presence in a frame  
        centroid_dict = dict() 						# Function creates a dictionary and calls it centroid_dict
        objectId = 0								# We inialize a variable called ObjectId and set it to 0
        for detection in detections:				# In this if statement, we filter all the detections for persons only
            # Check for the only person name tag 
            name_tag = detection[0]   # Coco file has string of all the names
            if name_tag == 'person':                
                x, y, w, h = detection[2][0],\
                            detection[2][1],\
                            detection[2][2],\
                            detection[2][3]      	# Store the center points of the detections
                xmin, ymin, xmax, ymax = convertBack(float(x), float(y), float(w), float(h))   # Convert from center coordinates to rectangular coordinates, We use floats to ensure the precision of the BBox            
                # Append center point of bbox for persons detected.
                centroid_dict[objectId] = (int(x), int(y), xmin, ymin, xmax, ymax) # Create dictionary of tuple with 'objectId' as the index center points and bbox
                objectId += 1 #Increment the index for each detection      
    
    # 3. Check which person bounding box are close to each other

        red_zone_list = [] # List containing which Object id is in under threshold distance condition. 
        red_line_list = []
        for (id1, p1), (id2, p2) in combinations(centroid_dict.items(), 2): # Get all the combinations of close detections, #List of multiple items - id1 1, points 2, 1,3
            dx, dy = p1[0] - p2[0], p1[1] - p2[1]  	# Check the difference between centroid x: 0, y :1
            distance = is_close(dx, dy) 			# Calculates the Euclidean distance
            if distance < 75.0:						# Set our social distance threshold - If they meet this condition then..
                if id1 not in red_zone_list:
                    red_zone_list.append(id1)       #  Add Id to a list
                    red_line_list.append(p1[0:2])   #  Add points to the list
                if id2 not in red_zone_list:
                    red_zone_list.append(id2)		# Same for the second id 
                    red_line_list.append(p2[0:2])
        
        for idx, box in centroid_dict.items():  # dict (1(key):red(value), 2 blue)  idx - key  box - value
            if idx in red_zone_list:   # if id is in red zone list
                cv2.rectangle(img, (box[2], box[3]), (box[4], box[5]), (255, 0, 0), 2) # Create Red bounding boxes  #starting point, ending point size of 2
            else:
                cv2.rectangle(img, (box[2], box[3]), (box[4], box[5]), (0, 255, 0), 2) # Create Green bounding boxes


    	# 3. Display risk analytics and risk indicators
    
        text = "No of at-risk people: %s" % str(len(red_zone_list)) 			# Count People at Risk
        location = (10,25)												# Set the location of the displayed text
        cv2.putText(img, text, location, cv2.FONT_HERSHEY_SIMPLEX, 1, (246,86,86), 2, cv2.LINE_AA)  # Display Text

        for check in range(0, len(red_line_list)-1):					# Draw line between nearby bboxes iterate through redlist items
            start_point = red_line_list[check] 
            end_point = red_line_list[check+1]
            check_line_x = abs(end_point[0] - start_point[0])   		# Calculate the line coordinates for x  
            check_line_y = abs(end_point[1] - start_point[1])			# Calculate the line coordinates for y
            if (check_line_x < 75) and (check_line_y < 25):				# If both are We check that the lines are below our threshold distance.
                cv2.line(img, start_point, end_point, (255, 0, 0), 2)   # Only above the threshold lines are displayed. 
        #=================================================================#
    return img

In [ ]:
netMain = None
metaMain = None 
altNames = None 

In [ ]:
def YOLO(videoname):
    """
    Perform Object detection
    """
    
    global metaMain, netMain, altNames, namesList
    configPath = "./cfg/yolov4.cfg"
    weightPath = "./yolov4.weights"
    metaPath = "./cfg/coco.data"
    if not os.path.exists(configPath):
        raise ValueError("Invalid config path `" +
                         os.path.abspath(configPath)+"`")
    if not os.path.exists(weightPath):
        raise ValueError("Invalid weight path `" +
                         os.path.abspath(weightPath)+"`")
    if not os.path.exists(metaPath):
        raise ValueError("Invalid data file path `" +
                         os.path.abspath(metaPath)+"`")
    if netMain is None:
        netMain = darknet.load_net_custom(configPath.encode(
            "ascii"), weightPath.encode("ascii"), 0, 1)  # batch size = 1
    if metaMain is None:
        metaMain = darknet.load_meta(metaPath.encode("ascii"))
    if altNames is None:
        try:
            with open(metaPath) as metaFH:
                metaContents = metaFH.read()
                import re
                match = re.search("names *= *(.*)$", metaContents, 
                                  re.IGNORECASE | re.MULTILINE)
                if match:
                    result = match.group(1)
                else:
                    result = None
                try:
                    if os.path.exists(result):
                        with open(result) as namesFH:
                            namesList = namesFH.read().strip().split("\n")
                            altNames = [x.strip() for x in namesList]
                except TypeError:
                    pass
        except Exception:
            pass
    
    uploadedvideo = videoname
    cap = cv2.VideoCapture(uploadedvideo)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    new_height, new_width = frame_height // 2, frame_width // 2
 
  # print("Video Reolution: ",(width, height))
    out = cv2.VideoWriter(
            "test2_output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 10.0,
            (new_width, new_height))
    
    # print("Starting the YOLO loop...")

    # Create an image we reuse for each detect
    darknet_image = darknet.make_image(new_width, new_height, 3)
    
    while True:
        prev_time = time.time()
        ret, frame_read = cap.read()
        # Check if frame present :: 'ret' returns True if frame present, otherwise break the loop.
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame_read, cv2.COLOR_BGR2RGB)
        frame_resized = cv2.resize(frame_rgb,
                                   (new_width, new_height),
                                   interpolation=cv2.INTER_LINEAR)

        darknet.copy_image_from_bytes(darknet_image,frame_resized.tobytes())

        detections = darknet.detect_image(netMain, namesList, darknet_image, thresh=0.25)
        image = cvBoxes(detections, frame_resized)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        print(1/(time.time()-prev_time))
        #cv2.imshow('Demo', image)
        cv2.waitKey(3)
        out.write(image)

    cap.release()
    out.release()
    print("Video Write Completed...")


    out = cv2.VideoCapture("test2_output.avi")  
    while(out.isOpened()):
      # Capture frame-by-frame
        ret, img = out.read()
        if ret == True:
            img = cv2.resize(img, (0,0), fx=0.5, fy=0.5) 
            frame = cv2.imencode('.jpg', img)[1].tobytes()
            yield (b'--frame\r\n'b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
            time.sleep(0.1) 
        else: 
            break
    frame_width = int(out.get(3))
    frame_height = int(out.get(4))
    new_height, new_width = frame_height // 2, frame_width // 2


In [ ]:
#Running the python file
!python app.py

In [ ]:
download('test2_output.avi')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Deployment**

Deploying the model through Flask using ngrok.

In [ ]:
#Checking the files
%ls 

 3rdparty/                image_yolov3.sh*         src/
 app.py                   image_yolov4.sh*         static/
 backup/                  include/                 templates/
 build/                   json_mjpeg_streams.sh*   test2_output.avi
 build.ps1                libdarknet.so            test.mp4
 camera.py                LICENSE                  Untitled
 cfg/                     Makefile                'Untitled (1)'
 cmake/                   net_cam_v3.sh*           uselib
 CMakeLists.txt           net_cam_v4.sh*           vcpkg.json
 darknet*                 obj/                     video_yolov3.sh*
 DarknetConfig.cmake.in   predictions.jpg          video_yolov4.sh*
 darknet_images.py        __pycache__/             vid.mp4
 darknet.py               README.md                yolov4.weights
 darknet_video.py         results/
 data/                    scripts/


In [ ]:
!pip install flask-ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask  

In [ ]:
import cv2
from flask import Flask, render_template, Response,request

app = Flask(__name__, template_folder='/content/gdrive/MyDrive/yolov4/darknet/templates')
run_with_ngrok(app)

@app.route('/')
def index():
    """Video streaming home page."""
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    video = "test.mp4"
    #video = "people.mp4"

    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(YOLO(video),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/website')
def website():
    """Video streaming home page."""
    return render_template('website.html')

@app.route('/about')
def about():
    """Video streaming home page."""
    return render_template('about.html')

@app.route('/covidmeasure')
def covidmeasure():
    """Video streaming home page."""
    return render_template('covidmeasure.html')


@app.route('/upload')
def upload():
    """Video streaming home page."""
    return render_template('upload.html') 

@app.route('/uploadedvideo')
def uploadedvideo():
  #video = "people.mp4" 
  #video = "check.mp4"
  video = "vid.mp4"
  return Response(YOLO(video),
                    mimetype='multipart/x-mixed-replace; boundary=frame')


@app.route('/live')
def live():
    """Video streaming home page."""
    return render_template('live.html')


@app.route('/checkLive')
def checkLive():
  video_width = 300
  video_path = record_video()    
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"

        


app.run();

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d0ec19b0c020.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [28/Jul/2021 09:13:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 09:13:29] "GET /docs/5.0/dist/css/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Jul/2021 09:13:30] "GET /static/1.1.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 09:13:30] "GET /docs/5.0/dist/css/bootstrap.min.css HTTP/1.1" 404 -


6.330633123082194
12.148986644035002
12.51119781412941
12.867581505649483
15.798290714186168
15.936834825957604
16.74386222644492
17.038934026649333
16.773391666699993
16.137524527721133
17.180898388537067
17.334126826687825
17.053898448427283
17.352916352924627
17.60685081017547
17.335989617346307
18.56700058875348
18.898113479587103
18.33872145439918
19.27502837736612
18.638286148500026
19.092876424237183
18.081078750883727
17.75877924651328
18.274162277090113
19.25927422502422
18.226278006639898
18.89113388131967


...

127.0.0.1 - - [28/Jul/2021 09:13:56] "GET /video_feed HTTP/1.1" 200 -


17.53508225506386
17.185685369871095
18.965449571567454
Video Write Completed...


127.0.0.1 - - [28/Jul/2021 09:14:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 09:14:08] "GET /docs/5.0/dist/css/bootstrap.min.css HTTP/1.1" 404 -


9.876666007328078
12.42977714556662
13.885343317873186
15.657515734774785
16.42146145895324
17.44718801996672
17.820953611094588
17.94654081322659
18.580078142304046
18.719473714747323
19.10313762462368
18.6075267624629
18.90978602923275
18.842759271322358
18.884754615038272
18.378980947540008
19.26812170102122
19.399124003866593
19.100962720756332
18.566425270464084
19.200732449815742
19.183520016831245
18.813431296032153
18.60265758333777
19.146739949146586
19.16222291259297
17.198228637034607
17.46666000366465
18.546557594516912
19.289921125853702
19.14560376858961
18.521905401168464
18.635801783468924
19.094354053045134
17.8523563061836
17.72575669210809
18.501887990965876
18.6265326695651
18.781671062471172
17.453213880000167
18.120292046485506
18.744152373461564
18.651049656932717
18.118178637304155
18.345780207763806
17.979698216735255
18.68536552768744
18.17628067620917
18.485253415601587
18.635967387199255
18.711874085442023
18.196862418436762
18.210925763509582
18.92685216104

127.0.0.1 - - [28/Jul/2021 09:14:20] "GET /video_feed HTTP/1.1" 200 -
